# Satellite Signal Acquisition – Galileo & GPS

**Generated:** 2025-04-29 05:41 UTC

This notebook contains the full GNU Octave code for the acquisition of Galileo E1‑C and GPS C/A signals, along with explanatory markdown cells and dedicated *student workspaces* where you can fill in the missing pieces.

## 1. Function definitions & visual explanations
The following helper functions are used by both acquisition scripts:

* **`CodeAcquisition`** – performs a 2‑D search in delay/Doppler space by correlating the received samples with a locally‑generated replica. The peak power after non‑coherent accumulation indicates a successful detection.  Mathematically, for each Doppler bin $f_d$ and code delay $\tau$ the coherent correlation over $N_c$ samples is
$$ R(f_d,\tau) = \sum_{n=0}^{N_c-1} r[n] \, s^*_{\mathrm{rep}}[n,\tau] \, e^{-j2\pi f_d n T_s}, $$
where $r[n]$ are the complex‑baseband samples, $s_{\mathrm{rep}}$ the code replica, and $T_s=1/f_s$ the sampling period.  The non‑coherent metric is obtained by accumulating $|R|^2$ over several coherent integrations.

* **`NoiseVarianceEstimator`** – estimates the thermal‑noise variance $\sigma_n^2$ from a correlation surface by removing the strongest peak and computing the sample variance of the remaining cells.

* **`InverseChiSquarePfa`** – provides the detection threshold $\gamma$ for a given probability of false alarm $P_{FA}$ under the assumption that the test statistic follows a scaled $\chi^2$ distribution with $2M$ degrees of freedom (where $M$ is the number of non‑coherent sums).  The threshold is
$$ \gamma = 2\,\sigma_n^2 \; F^{-1}_{\chi^2_{2M}}\bigl(1-P_{FA}\bigr). $$

Feel free to step through the code to see the exact implementation details.

In [ ]:
%% CodeAcquisition.m
function sspace = CodeAcquisition(signal, rep, Nc, Doppler_vec, fs, fi)
% Purpose.
% Evaluate the search space for the code acquisition using the time domain FFT technique
%
% Syntax.
% sspace = DftParallelCodePhaseAcquisition(sig, locC, N, dstep, DopStep)
%
% Input Parameters.
% sig       : [vector] the Galileo/GPS input signal, corrupted by Doppler
%              shift, code delay, noise and eventually interferer
% locC      : [vector] the local code replica
%
% N         : [integer] the input signal and local code length
%
% Nd        : [integer] number of Doppler bin used for the search space
%
% DopStep   : [Hz] Doppler bin width in Hz
%
% fs        : [Hz] sampling frequency
%
% fi        : [Hz] intermediate frequency
%
% Output Parameters.
% sspace       : [matrix] the search space


fif = fi/fs;    % normalized intermediate frequency
Doppler_vec_norm = Doppler_vec/fs;    % normalized Doppler step

sspace = zeros(size(Doppler_vec_norm,2), size(signal,2));

%#########################################################################
% Complete here: Compute the complex conjugate of the FFT of the replica
% Remember: 
%     - For computational efficiency we prepare the complex conjugate of the FFT of the replica only once here
% What you need to do:
%     - first step: calculate the fft of the replica rep - use the fft() function here
%     - second step: calculate the complex conjugate of the result of step 1 - use the conj() function here
%     - complete the following command below this guidance: F_rep = 
%#########################################################################



t = 0:(size(signal,2) - 1);  % time index

for i = 1:length(Doppler_vec_norm),
     fc = fif + Doppler_vec_norm(i);
%     fc = fif + (i - ceil(length(Doppler_vec_norm)/2))*Doppler_vec_norm;
    %#########################################################################
    % Complete here: Add the Doppler to the signal samples
    % Remember: 
    %     - Acquisition process is a search over Delay and Frequency. The time delay is efficiently evaluated using the fft
    %     - For the Frequency domain we need to sequentially cover all identified Doppler bins.
    %     - shifting a complex signal in the frequency domain corresponds to multiplying with a complex exponential function
    % What you need to do:
    %     - Multiply (using the .* function) with the complex exponential of (-2i*pi*fc.*t)
    %     - complete the following command below this guidance: IQ_comp = 
    %#########################################################################
    
    
    X = fft(IQ_comp);
    sspace(i,:) = ifft(X.*F_rep);
end

sspace = real(sspace.*conj(sspace));





% sspace_1ms = reshape(sspace_abs,[size(sspace,1),2046,8]);
% sspace_1ms_sum = squeeze(sum(sspace_1ms,3));
% 
% sspace_1ms_B = reshape(sspace,[size(sspace,1),2046,8]);
% sspace_1ms_sum_B = squeeze(sum(sspace_1ms_B,3)).*conj(squeeze(sum(sspace_1ms_B,3)));
% 
% 
% 
% figure;
% surf( sspace_1ms_sum, 'EdgeColor', 'none');
%       set( gca, 'FontSize', 16)
%       xlabel('Delay [samples]');
%       ylabel('Doppler');

In [ ]:
%% NoiseVarianceEstimator.m
function sigma_est = NoiseVarianceEstimator( y, fs, fc )


% Arguments:
%   y :         [vector] contains the input samples
%   fs:         [scalar] sampling frequency
%   fc:         [scalar] code rate

%
% Returns:
%   sigma_est:     [scalar] the estimated noise variance
%%

% First generate a fictitious code
clen = round( length( y ) / fs * fc );
code =  sign( rand( 1, clen ) - 0.5 );      % A bipolar random code usually has
                                            % good correlation properties

% Resample the code
loc = reshape(repmat(code,fs/fc,1),1,fs/fc*size(code,2));

% Now compute the correlators (for a single Doppler value is enough )
correlators = CodeAcquisition( y, loc, 1, 0, fs, 0 );


% Down-sample them to get uncorrelated values:
step = round( fs / fc );
correlators = correlators(1:step:end);

% Finally my noise variance estimate
sigma_est = mean( correlators ) / 2;


In [ ]:
%% InverseChiSquarePfa.m
function Th = InverseChiSquarePfa( K, PfaT )

Th = -2*log( PfaT );

Niter = 1000;
Pfa = 0;
ii = 1;

fact = factorial( K - 1 );

while abs( PfaT - Pfa ) > 1e-3 * PfaT,
    Pfa = gammainc( Th / 2, K, 'upper' );					% Compute the ccdf
    pdf = exp( -Th / 2 ) .* ( Th / 2 ).^( K - 1 ) / fact;	% Compute the pdf

    Th = Th + ( Pfa - PfaT )/ pdf;

    ii = ii + 1;
    if ii > Niter,
        break;
    end
end


## 2. Galileo E1‑C acquisition
The script below (`acquire_galileo_e1c.m`) performs acquisition of a Galileo E1‑C signal.  Two areas have been intentionally left for you to complete:

1. **Signal‑parameter configuration** – define the chip rate $f_c$, sample rate $f_s$, and the primary‑code period $T_\text{code}$.  
2. **BOC subcarrier generation** – generate the BOC(1,1) square‑wave subcarrier and overlay it on the code replica.

Read the inline comments carefully and then complete the corresponding *Student workspace* cells that follow the script.

In [ ]:
%% acquire_galileo_e1c.m
clc
clear;
##close all;
%=========================================================================%
% Load the prepared IQ samples                                            %
%=========================================================================%
load IQ_samples_working_e1c.mat

%=========================================================================%
% Load the PRN  code                                                  %
%=========================================================================%

load E1_codes.mat
PRN = E1C;

%=========================================================================%
% Resample the replica                                                    %
%=========================================================================%

%#########################################################################
% Complete here: Configure for Galileo E1-C signal parameters
% Remember:
%     - the lecture material from this morning does provide you with the necessary information
% What you need to do:
%     - specify the chip rate of E1-C in Hz: fc =
%     - specify the sample rate of the recorded signal in Hz, check for this the GNURadio config: fs =
%     - specify the primary code period of E1-C in ms: T_Primary code =
%#########################################################################

N_noncoh = 1; %Number of non-coherent accumulations
Tdwell = Tcoh *N_noncoh; %Total Dwell time [ms]
Nsample = Tcoh * fs; %Number of signal samples in Coherent Integration Period

PRN = repmat(PRN,1,round(Tcoh/T_primary_code));

%=========================================================================%
% Build up the replica                                                 %
%=========================================================================%
for i=1:size(PRN,1)
  Rep(i,:) = reshape(repmat(PRN(i,:),fs/fc,1),1,fs/fc*size(PRN,2));
end

%#########################################################################
% Complete here: Generate the BOC subcarrier for the replica
% Remember:
%     - Galileo E1C is modulated as a BOC(1,1) therefore we need to generate for the replica signal
%     - You need to overlay (multiply) every PRN code chip with the corresponding BOC spreading symbol
% What you need to do:
%     - Step 1: generate a single BOC spreading symbol at the correct sample rate. Hint: considering
%               the ratio of sample rate and chip rate you need to generate a BOC spreading symbol of 8 bits.
%               BOC1_1_subcarrier = ...
%     - Step 2: repeat the generated spreading symbol (BOC1_1_subcarrier) 4092 times to align with the number of chips in the replica that we are using
%               BOC1_1_subcarrier_4ms = ....
%     - Step 3: BOC1_1_subcarrier_4ms is a vecotr of size 1 x 4092*8. Now we do repeat this vector 50 times (this is the number of PRN sequences we have loaded)
%               use for this the repmat function. BOC1_1_subcarrier_4ms = ...
%     - Step 4: Now we do generate the final replica of all PRN sequences. Do a element by element multiplication (us for this the operator .* )
%               of the already existing variabl Rep and the variable BOC1_1_subcarreir_4ms
%               Rep = ....
%#########################################################################




%=========================================================================%
% Acquisition Process                                                     %
%=========================================================================%

Doppler_range = 6e3 ;
Doppler_step = round(2/(3*Tdwell));
Doppler_vec = Doppler_step:Doppler_step:Doppler_range;
Doppler_vec = [-fliplr(Doppler_vec) 0 Doppler_vec];


%=========================================================================%
% Search over all possible PRN sequences                                  %
%=========================================================================%
for code_select = 10:10
Rep_search = Rep(code_select,:);

Nc = round(Tcoh/1e-3)*fs/1e3;
sspace = 0;
for ii = 1:N_noncoh
    y =  data( (ii - 1) * Nc + (1:2*Nc) ).';   % use just 1 period of signal at the time

    % Compute the search space for a single coherent integration epoch
    Tsspace = CodeAcquisition( y, [Rep_search zeros(1,size(Rep_search,2))], Nc, Doppler_vec, fs, fi);
    sspace = sspace + Tsspace;  % Non-coherently accumulate the results

end

%=========================================================================%
% Decision Logic                                                          %
%=========================================================================%
Pfa_global = 0.1;
Pfa_cell = 1-(1-Pfa_global).^(1/numel(sspace));

Th = InverseChiSquarePfa(N_noncoh, Pfa_cell);
sigma_est = NoiseVarianceEstimator(y, fs, fc);
Th = sigma_est * Th;

##sspace_single_period = reshape(sspace,[size(sspace,1),2*fs/fc*size(PRN,2)/(Tcoh*1e3),2*Tcoh*1e3]);
##sspace_single_period = squeeze(sum(sspace_single_period,3));


if (max(max(sspace)) > Th)
       [doppler_id, delay_id] = find(sspace == max(max(sspace)));
       fprintf('PRN %i acquired, Doppler [Hz]: %f, Code Delay [samples]: %i\n',code_select, Doppler_vec(doppler_id(1)), delay_id(1));
        figure
          surf( sspace, 'EdgeColor', 'none');
          set( gca, 'FontSize', 16)
          xlabel('Delay [samples]');
          ylabel('Doppler Bin');
          xlim([0 size(sspace,2)]);
          ylim([1 length(Doppler_vec)])

end


end




### ✏️ Student workspace – Task G1: Configure Galileo signal parameters
Using information from the lecture (chip rate, sample rate, code period), assign values to the variables below.

In [ ]:
%% Task G1
% --- Configure Galileo E1‑C parameters here
fc = ;               % [Hz] chip rate
fs = ;               % [Hz] sample rate
T_primary_code = ;   % [ms] primary‑code period

### ✏️ Student workspace – Task G2: Generate BOC(1,1) subcarrier
Create a vector `boc` of length equal to one PRN‑code chip at the sampling rate `fs` and values ±1 corresponding to the BOC(1,1) modulation.

In [ ]:
%% Task G2
% --- Generate the BOC(1,1) subcarrier here
% boc = ...

## 3. GPS L1 C/A acquisition
The script below (`acquire_gps_ca.m`) acquires the GPS C/A signal.  Two student tasks have been left open:

1. **Replica up‑sampling** – up‑sample the 1.023 MHz C/A code to the 2.046 MHz sampling rate.
2. **Doppler‑vector definition** – create a symmetrical Doppler search vector covering ±10 kHz with a step of ~667 Hz.

Complete them in the workspaces that follow.

In [ ]:
%% acquire_gps_ca.m
clc
clear;
##close all;
%=========================================================================%
% Load the prepared IQ samples                                            %
%=========================================================================%
load IQ_samples.mat

%=========================================================================%
% Load the PRN  code                                                  %
%=========================================================================%

load gps_ca_codes.mat;

%=========================================================================%
% Resample the replica                                                    %
%=========================================================================%

fc = 1.023e6; % GNSS signal chip rate
fs = 2.046e6; % Sample rate
Tcoh = 5e-3; % Coherent Integration time [ms]
T_primary_code = 1e-3; %Period of the primary code [ms]
N_noncoh = 1; %Number of non-coherent accumulations
Tdwell = Tcoh *N_noncoh; %Total Dwell time [ms]
Nsample = Tcoh * fs; %Number of signal samples in Coherent Integration Period

PRN = repmat(gps_ca,1,round(Tcoh/T_primary_code));

%=========================================================================%
% Build up the replica                                                 %
%=========================================================================%
for i=1:size(PRN,1)
%#########################################################################
% Complete here: Build up the replica
% Remember: 
%     - the GPS C/A is a BPSK(1), i.e. a signal at a rate of 1.023 MHz
%     - the recorded signal is sampled at a rate of 2.046 MHz
% What you need to do:
%     - up-sample the C/A code replica to 2.046 MHz, i.e. repeat every chip of the C/A code replica 2 times
%     - you can use the functions repmat and reshape (ref. homework)
%     - complete the following command below this guidance: Rep(i,:) = 
%#########################################################################
 
end


%=========================================================================%
% Acquisition Process                                                     %
%=========================================================================%

Doppler_range = 10e3 ;
Doppler_step = round(2/(3*Tdwell));

%#########################################################################
% Complete here: Set up the Doppler Vector
% Remember: 
%     - Doppler for a static user can be in the range [-10 kHz ... 10 kHz] (with good margin)
% What you need to do:
%     - build up a vector (single row, N columns) that ranges from -Doppler_range in steps of Doppler_step up to +Doppler_range
%     - for symmetry purpuses it is good to center around 0 Hz, you do not need to match exactly +/- 10 kHz on the lower/upper limit
%     - complete the following command below this guidance: Doppler_vec = 
%#########################################################################




%=========================================================================%
% Search over all possible PRN sequences                                  %
%=========================================================================%
for code_select = 1:20
Rep_search = Rep(code_select,:);

Nc = round(Tcoh/1e-3)*fs/1e3;
sspace = 0;
for ii = 1:N_noncoh
    y =  data( (ii - 1) * Nc + (1:Nc) ).';   % use just 1 period of signal at the time

    % Compute the search space for a single coherent integration epoch
    Tsspace = CodeAcquisition( y, Rep_search, Nc, Doppler_vec, fs, fi);
    sspace = sspace + Tsspace;  % Non-coherently accumulate the results

end

%=========================================================================%
% Decision Logic                                                          %
%=========================================================================%
Pfa_global = 0.1;
Pfa_cell = 1-(1-Pfa_global).^(1/numel(sspace));

Th = InverseChiSquarePfa(N_noncoh, Pfa_cell);
sigma_est = NoiseVarianceEstimator(y, fs, fc);
Th = sigma_est * Th;

sspace_single_period = reshape(sspace,[size(sspace,1),fs/fc*size(PRN,2)/(Tcoh*1e3),Tcoh*1e3]);
sspace_single_period = squeeze(sum(sspace_single_period,3));


if (max(max(sspace)) > Th)
       [doppler_id, delay_id] = find(sspace == max(max(sspace)));
       fprintf('PRN %i acquired, Doppler [Hz]: %f, Code Delay [samples]: %i\n',code_select, Doppler_vec(doppler_id(1)), delay_id(1));
        figure
          surf( sspace_single_period, 'EdgeColor', 'none');
          set( gca, 'FontSize', 16)
          xlabel('Delay [samples]');
          ylabel('Doppler Bin');
          xlim([0 size(sspace_single_period,2)]);
          ylim([1 length(Doppler_vec)])

end


end




### ✏️ Student workspace – Task P1: Build the up‑sampled GPS C/A replica

In [ ]:
%% Task P1
% --- Up‑sample the PRN code replica here
% Rep = ...

### ✏️ Student workspace – Task P2: Define the Doppler search vector

In [ ]:
%% Task P2
% --- Create the Doppler vector here
% Doppler_vec = ...